In [68]:
import pandas as pd
import numpy as np
import sys
import sklearn

In [69]:
df = pd.read_csv('.../kddpretrainPre.csv')
df_test = pd.read_csv('.../kddpretestPre.csv')

In [70]:
X_train = df.drop('label',1)
Y_train = df.label

X_test = df_test.drop('label',1)
Y_test = df_test.label

In [71]:
colNames=list(X_train)
colNames_test=list(X_test)

In [72]:
from sklearn import preprocessing

scaler0 = preprocessing.StandardScaler().fit(X_train)
X_train=scaler0.transform(X_train)

scaler1 = preprocessing.StandardScaler().fit(X_test)
X_test=scaler1.transform(X_test) 

In [73]:
print(X_train.shape)

(125973, 122)


In [74]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.under_sampling import RandomUnderSampler 
sampling_strategy = {0: 6, 1: 12, 2: 18, 3: 24, 4: 30}
rus1 = RandomUnderSampler(sampling_strategy = sampling_strategy, random_state=99, replacement=False)

In [75]:
X_train1, Y_train1 = rus1.fit_sample(X_train,Y_train)

In [76]:
from sklearn.feature_selection import SelectPercentile, f_classif
np.seterr(divide='ignore', invalid='ignore');
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion="gini",max_depth=4,random_state=0)
rfe = RFE(estimator=clf,n_features_to_select=17, step=1)

In [77]:
rfe.fit(X_train1, Y_train1)
X_train1=rfe.transform(X_train1)
true=rfe.support_
rfecolindex_train=[i for i, x in enumerate(true) if x]
rfecolname_train=list(colNames[i] for i in rfecolindex_train)
X_train1.shape

(90, 17)

In [78]:
X_test=X_test[:,rfecolindex_train]

In [79]:
Dtree1=DecisionTreeClassifier(max_depth = 4,random_state=0)
Dtree1.fit(X_train1, Y_train1)

DecisionTreeClassifier(max_depth=4, random_state=0)

In [81]:
Y_pred=Dtree1.predict(X_test)

pd.crosstab(Y_test, Y_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

Predicted attacks,0,1,2,3,4
Actual attacks,,,,,
0,2833,931,1413,4316,218
1,357,5051,1264,786,2
2,0,777,1509,135,0
3,4,1,1585,1264,31
4,0,0,10,23,34


In [82]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [83]:
accuracy = cross_val_score(Dtree1, X_test, Y_test, cv=5, scoring='accuracy')
print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))

Accuracy: 0.84932 (+/- 0.01680)


In [84]:
sampling_strategy = {0: 12, 1: 24}
rus2 = RandomUnderSampler(sampling_strategy = sampling_strategy, random_state=99, replacement=False)

In [85]:
X_train2, Y_train2 = rus2.fit_sample(X_train,Y_train)

In [86]:
rfe.fit(X_train2, Y_train2)
X_train2=rfe.transform(X_train2)
true=rfe.support_
rfecolindex_train=[i for i, x in enumerate(true) if x]
rfecolname_train=list(colNames[i] for i in rfecolindex_train)
X_train2.shape

(12739, 17)

In [87]:
Dtree2=DecisionTreeClassifier(max_depth = 4,random_state=0)
Dtree2.fit(X_train2, Y_train2)

DecisionTreeClassifier(max_depth=4, random_state=0)

In [88]:
Y_pred=Dtree2.predict(X_test)

pd.crosstab(Y_test, Y_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

Predicted attacks,0,1,2,3
Actual attacks,,,,
0,409,0,9273,29
1,324,15,7104,17
2,598,1,1822,0
3,1,0,2884,0
4,0,0,67,0


In [89]:
accuracy = cross_val_score(Dtree2, X_test, Y_test, cv=30, scoring='accuracy')
print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))

Accuracy: 0.84989 (+/- 0.02495)


In [90]:
sampling_strategy = {0: 1, 1: 2, 2: 3, 3: 4,}
rus3 = RandomUnderSampler(sampling_strategy = sampling_strategy, random_state=99, replacement=False)

In [91]:
X_train3, Y_train3 = rus3.fit_sample(X_train,Y_train)

In [92]:
rfe.fit(X_train3, Y_train3)
X_train3=rfe.transform(X_train3)
true=rfe.support_
rfecolindex_train=[i for i, x in enumerate(true) if x]
rfecolname_train=list(colNames[i] for i in rfecolindex_train)
X_train3.shape

(62, 17)

In [93]:
Dtree3=DecisionTreeClassifier(max_depth = 4,random_state=0)
Dtree3.fit(X_train3, Y_train3)

DecisionTreeClassifier(max_depth=4, random_state=0)

In [94]:
Y_pred=Dtree3.predict(X_test)

pd.crosstab(Y_test, Y_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

Predicted attacks,2,3,4
Actual attacks,,,
0,6648,9,3054
1,7101,0,359
2,2421,0,0
3,2849,1,35
4,31,2,34


In [95]:
accuracy = cross_val_score(Dtree3, X_test, Y_test, cv=5, scoring='accuracy')
print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))

Accuracy: 0.84932 (+/- 0.01680)


In [96]:
print("Mean of Whole Accuracy = ")
print(((0.84932 + 0.84989 + 0.84932)/3))

Mean of Whole Accuracy = 
0.84951
